## Title: Using the Google API to get University Addresses
### Author: Meenakshi Parameshwaran
### Date: 05/04/16

Note that this assigment was written in Python 3.4

University UKPRN-Name lookup from here: https://www.google.co.uk/url?sa=t&rct=j&q=&esrc=s&source=web&cd=2&ved=0ahUKEwiAzLveqPjLAhVDgg8KHYx1Dj4QFgghMAE&url=https%3A%2F%2Fwww.hesa.ac.uk%2Fdox%2Funistats%2FUNISTATS_UKPRN_lookup_20141030.xls&usg=AFQjCNG1-YuYokBm7hPDea_vo9ua3-g0pg&sig2=mY5ezIiwILVyfvLPkUrANQ&bvm=bv.118443451,d.ZWU

In [1]:
import csv, pandas, urllib, json, os

/Users/meenaparam/anaconda/lib/python3.4/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
os.getcwd()

'/Users/meenaparam/Documents/Python_work/Unis'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'unilookup.csv',
 'unis_geocoding.ipynb',
 'UNISTATS_UKPRN_lookup_20141030.xls']

In [4]:
unis = pandas.read_csv('unilookup.csv')

In [5]:
unis.head(n=10)

,UKPRN,NAME
0,10000047,Association of British Dispensing Opticians
1,10000055,Abingdon and Witney College
2,10000080,Access to Music
3,10000093,Accrington and Rossendale College
4,10000143,Barking and Dagenham London Borough Council
5,10000163,AECC Chiropractic College
6,10000248,ALRA (Academy of Live and Recorded Arts)
7,10000256,Alton College
8,10000275,Amersham and Wycombe College
9,10000291,Anglia Ruskin University


In [6]:
repl = " UK"
unis['longname'] = unis['NAME'] + repl

In [7]:
smallunis = unis[0:10]

In [8]:
print(smallunis)
type(smallunis)

      UKPRN                                         NAME  \
0  10000047  Association of British Dispensing Opticians   
1  10000055                  Abingdon and Witney College   
2  10000080                              Access to Music   
3  10000093            Accrington and Rossendale College   
4  10000143  Barking and Dagenham London Borough Council   
5  10000163                    AECC Chiropractic College   
6  10000248     ALRA (Academy of Live and Recorded Arts)   
7  10000256                                Alton College   
8  10000275                 Amersham and Wycombe College   
9  10000291                     Anglia Ruskin University   

                                         longname  
0  Association of British Dispensing Opticians UK  
1                  Abingdon and Witney College UK  
2                              Access to Music UK  
3            Accrington and Rossendale College UK  
4  Barking and Dagenham London Borough Council UK  
5                    AECC C

pandas.core.frame.DataFrame

In [9]:
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/json?'

In [10]:
uni_name = []
formatted_address = []
long_name = []
short_name = []
first_type = []
second_type = []

In [11]:
for a in unis['longname']:
    address = a
    if len(address) < 1 : break

    url = serviceurl + urllib.parse.urlencode({'sensor':'false', 'address': address})
    print ('Retrieving', url)

    urlopened = urllib.request.urlopen(url) 

    # need to decode it for Python 3
    mydata = urlopened.readall().decode('utf-8')

    print ('Retrieved',len(mydata),'characters')

    try: jsoutput = json.loads(str(mydata))
    except: jsoutput = None
    if 'status' not in jsoutput or jsoutput['status'] != 'OK':
        print ('==== Failure To Retrieve ====')
        print (mydata)
        continue
    
#     print (json.dumps(jsoutput, indent=4))
    
    for result in jsoutput['results']:
        n = 0
        for component in result['address_components']:
            uni_name.append(address)
            formatted_address.append(result['formatted_address'])
            long_name.append(result['address_components'][n]['long_name'])
            short_name.append(result['address_components'][n]['short_name'])
            first_type.append(result['address_components'][n]['types'][0])
            try: second_type.append(result['address_components'][n]['types'][1])
            except: second_type.append("None")
            n = n + 1


#     for component in jsoutput['results'][0]['address_components']:
#         if component['types'] == "postal_code":
#             long_name.append(component['long_name'])
#             short_name.append(component['short_name'])
#             types.append(component['types'])

#     formatted_address.append(jsoutput['results'][0]['formatted_address'])
#     formatted_address.append(jsoutput['results'][0]['partial_match'])

Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=Association+of+British+Dispensing+Opticians+UK
Retrieved 52 characters
==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=Abingdon+and+Witney+College+UK
Retrieved 5822 characters
Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=Access+to+Music+UK
Retrieved 52 characters
==== Failure To Retrieve ====
{
   "results" : [],
   "status" : "ZERO_RESULTS"
}

Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=Accrington+and+Rossendale+College+UK
Retrieved 2095 characters
Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=Barking+and+Dagenham+London+Borough+Council+UK
Retrieved 2153 characters
Retrieving http://maps.googleapis.com/maps/api/geocode/json?sensor=false&address=AECC+Chiropractic+College+UK
Retrieved 52

In [12]:
newdf = pandas.DataFrame([uni_name,formatted_address, long_name,short_name,first_type,second_type]).T

In [13]:
newdf.columns = ['uni_name','formatted_address', 'long_name','short_name','first_type','second_type']

In [14]:
newdf

,uni_name,formatted_address,long_name,short_name,first_type,second_type
0,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",Abingdon,Abingdon,locality,political
1,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",Abingdon,Abingdon,postal_town,None
2,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",Vale of White Horse District,Vale of White Horse District,administrative_area_level_3,political
3,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",Oxfordshire,Oxfordshire,administrative_area_level_2,political
4,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",England,England,administrative_area_level_1,political
5,Abingdon and Witney College UK,"Abingdon, Abingdon, Oxfordshire, UK",United Kingdom,GB,country,political
6,Abingdon and Witney College UK,"Abingdon, VA, USA",Abingdon,Abingdon,locality,political
7,Abingdon and Witney College UK,"Abingdon, VA, USA",Washington County,Washington County,administrative_area_level_2,political
8,Abingdon and Witney College UK,"Abingdon, VA, USA",Virginia,VA,administrative_area_level_1,political
9,Abingdon and Witney College UK,"Abingdon, VA, USA",United States,US,country,political


In [15]:
postcodes = newdf[newdf['first_type'] == "postal_code" ]

,uni_name,formatted_address,long_name,short_name,first_type,second_type
20,Accrington and Rossendale College UK,"College St, Accrington, Lancashire BB5 1TG, UK",BB5 1TG,BB5 1TG,postal_code,None
28,Barking and Dagenham London Borough Council UK,"Greenwich Council Estate Office, 140 Eynsham D...",SE2 9PT,SE2 9PT,postal_code,None
34,Alton College UK,"College Rd, Denstone, Uttoxeter, Staffordshire...",ST14 5HR,ST14 5HR,postal_code,None
46,Anglia Ruskin University UK,"Anglia Ruskin University, Brentwood, Essex CM1...",CM15 9FE,CM15 9FE,postal_code,None
51,Bournemouth University UK,"University Roundabout, Bournemouth, Poole BH12...",BH12 5EQ,BH12 5EQ,postal_code,None
61,Bradford College UK,"Bradford College Old Building, Bradford BD7 1D...",BD7 1DP,BD7 1DP,postal_code,None
74,The University of Brighton UK,"C-University of Brighton, Grand Parade, Bright...",BN2 0JY,BN2 0JY,postal_code,None
88,British School of Osteopathy. The UK,"European School of Osteopathy, 104 Tonbridge R...",ME16 8SL,ME16 8SL,postal_code,None
108,City and Islington College UK,"City and Islington College (Junction Road), Lo...",N19 5RD,N19 5RD,postal_code,None
114,City and Islington College UK,"City and Islington College (Centre for Health,...",N7 0RY,N7 0RY,postal_code,None


In [16]:
newdf["first_type"].describe()

count        2082
unique         17
top       country
freq          341
Name: first_type, dtype: object

In [33]:
postcodes = newdf[newdf['first_type'] == "postal_code" ]

In [34]:
postcodes.describe()

,uni_name,formatted_address,long_name,short_name,first_type,second_type
count,202,202,202,202,202,202
unique,101,187,184,184,1,1
top,"Manchester College, UK","North West Kent College, Dering Way, Lower Hig...",B32 3NT,B32 3NT,postal_code,None
freq,9,2,2,2,202,202


In [102]:
postcodes_new = postcodes[['uni_name', 'formatted_address', 'long_name']]

In [103]:
postcodes_new.columns = ['uni_name', 'full_address', 'postcode']

In [104]:
# postcodes_new

In [105]:
import re

In [107]:
postcodes_new['uni_name_fixed'] = postcodes_new['uni_name'].map(lambda x: x.rstrip('[,]\sUK'))

/Users/meenaparam/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [108]:
postcodes_new['uni_name_fixed'] = postcodes_new['uni_name_fixed'].map(lambda x: x.rstrip(','))

/Users/meenaparam/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [109]:
postcodes_new

,uni_name,full_address,postcode,uni_name_fixed
20,Accrington and Rossendale College UK,"College St, Accrington, Lancashire BB5 1TG, UK",BB5 1TG,Accrington and Rossendale College
28,Barking and Dagenham London Borough Council UK,"Greenwich Council Estate Office, 140 Eynsham D...",SE2 9PT,Barking and Dagenham London Borough Council
34,Alton College UK,"College Rd, Denstone, Uttoxeter, Staffordshire...",ST14 5HR,Alton College
46,Anglia Ruskin University UK,"Anglia Ruskin University, Brentwood, Essex CM1...",CM15 9FE,Anglia Ruskin University
51,Bournemouth University UK,"University Roundabout, Bournemouth, Poole BH12...",BH12 5EQ,Bournemouth University
61,Bradford College UK,"Bradford College Old Building, Bradford BD7 1D...",BD7 1DP,Bradford College
74,The University of Brighton UK,"C-University of Brighton, Grand Parade, Bright...",BN2 0JY,The University of Brighton
88,British School of Osteopathy. The UK,"European School of Osteopathy, 104 Tonbridge R...",ME16 8SL,British School of Osteopathy. The
108,City and Islington College UK,"City and Islington College (Junction Road), Lo...",N19 5RD,City and Islington College
114,City and Islington College UK,"City and Islington College (Centre for Health,...",N7 0RY,City and Islington College


In [88]:
postcodes_final = postcodes_new[['uni_name_fixed', 'full_address', 'postcode']]

In [89]:
postcodes_final

,uni_name_fixed,full_address,postcode
20,Accrington and Rossendale College,"College St, Accrington, Lancashire BB5 1TG, UK",BB5 1TG
28,Barking and Dagenham London Borough Council,"Greenwich Council Estate Office, 140 Eynsham D...",SE2 9PT
34,Alton College,"College Rd, Denstone, Uttoxeter, Staffordshire...",ST14 5HR
46,Anglia Ruskin University,"Anglia Ruskin University, Brentwood, Essex CM1...",CM15 9FE
51,Bournemouth University,"University Roundabout, Bournemouth, Poole BH12...",BH12 5EQ
61,Bradford College,"Bradford College Old Building, Bradford BD7 1D...",BD7 1DP
74,The University of Brighton,"C-University of Brighton, Grand Parade, Bright...",BN2 0JY
88,British School of Osteopathy. The,"European School of Osteopathy, 104 Tonbridge R...",ME16 8SL
108,City and Islington College,"City and Islington College (Junction Road), Lo...",N19 5RD
114,City and Islington College,"City and Islington College (Centre for Health,...",N7 0RY


In [63]:
postcodes_final.to_csv('postcodes_final.csv')

In [64]:
os.listdir()

['.ipynb_checkpoints',
 'postcodes_final.csv',
 'unilookup.csv',
 'unis_geocoding.ipynb',
 'UNISTATS_UKPRN_lookup_20141030.xls']